In [2]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [3]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Import the Imagery of Nighttime Lights (Harmonized DMSP-OLS and VIIRS)
dem = ee.Image("USGS/SRTMGL1_003") 

# Clip the images to Colombia's boundaries
dem_COL = dem.clip(colombiaMpios)


In [4]:
# Calculate the mean elevation for the current year in square meters
def calculate_dem_mean(feature):

    # Use reduceRegion to calculate the mean elevation within the feature's geometry
    mean_dem = dem_COL.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('elevation') 

    # Return a new feature with only the 'mean_dem' and 'codmpio' properties
    return ee.Feature(None, {
        'mean_dem': mean_dem,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

# Dictionary to store results for each year
results_dict1 = {}

# Use a lambda function to pass the current year into the map function
mean_dem_all = colombiaMpios.map(lambda feature: calculate_dem_mean(feature))
results_dict1['mean_dem'] = mean_dem_all

# Optionally print the first feature's info to check
print(results_dict1['mean_dem'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'mean_dem': 1944.8130860917647}}


In [5]:
# Calculate the mean ruggedness (Terrain Ruggedness Index) for each feature
def calculate_ruggedness_mean(feature):
    
    # Calculate TRI using focal statistics (standard deviation of elevation)
    # TRI measures the difference between a cell and its surrounding cells
    ruggedness = dem_COL.select('elevation').reduceNeighborhood(
        reducer=ee.Reducer.stdDev(),
        kernel=ee.Kernel.square(1)  # 3x3 pixel window
    )
    
    # Use reduceRegion to calculate the mean ruggedness within the feature's geometry
    mean_ruggedness = ruggedness.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('elevation_stdDev')  # Band name changes to 'elevation_stdDev' after reduceNeighborhood

    # Return a new feature with only the 'mean_ruggedness' and 'codmpio' properties
    return ee.Feature(None, {
        'mean_ruggedness': mean_ruggedness,
        'codmpio': feature.get('codmpio'),  # Ensure 'codmpio' exists in the original feature
    })

results_dict2 = {}

# Use a lambda function to calculate ruggedness for all features
mean_ruggedness_all = colombiaMpios.map(lambda feature: calculate_ruggedness_mean(feature))
results_dict2['mean_ruggedness'] = mean_ruggedness_all

# Optionally print the first feature's info to check
print(results_dict2['mean_ruggedness'].first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'codmpio': 5002, 'mean_ruggedness': 10.996409902363228}}


In [6]:
# Set description
description_file = 'Elevation'

# Export the FeatureCollection to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=results_dict1['mean_dem'],
    description=description_file,
    folder="EarthEngineExports",
    fileFormat='CSV'
)

# Start the export task
export_task.start()
print(f"Exporting {description_file} to Google Drive...")

Exporting Elevation to Google Drive...


In [7]:
# Set description
description_file = 'Ruggedness'

# Export the FeatureCollection to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=results_dict2['mean_ruggedness'],
    description=description_file,
    folder="EarthEngineExports",
    fileFormat='CSV'
)

# Start the export task
export_task.start()
print(f"Exporting {description_file} to Google Drive...")

Exporting Ruggedness to Google Drive...
